In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Program Files\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Program Files\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Program Files\Python310\li

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Program Files\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Program Files\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Program Files\Python310\li

AttributeError: _ARRAY_API not found

In [4]:
# Khởi tạo Spark session
spark = SparkSession.builder.appName("CollaborativeFiltering").getOrCreate()

In [5]:
# Đọc dữ liệu
hotel_comments_df = pd.read_csv("data/hotel_comments_cleaned.csv")
hotel_info_df = pd.read_csv("data/hotel_info_cleaned.csv")

In [6]:
# Tiền xử lý dữ liệu cho mô hình PySpark ALS
als_data = hotel_comments_df[['Hotel ID', 'Reviewer ID', 'Score']]
als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))

C:\Users\admin\AppData\Local\Temp\ipykernel_15012\1463577058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
C:\Users\admin\AppData\Local\Temp\ipykernel_15012\1463577058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))


In [7]:
# Chuyển đổi DataFrame sang Spark DataFrame
spark_df = spark.createDataFrame(als_data)

In [8]:
# Chia dữ liệu thành tập đào tạo và kiểm tra
(training, test) = spark_df.randomSplit([0.8, 0.2])

In [9]:
# Xây dựng mô hình ALS
als = ALS(maxIter=10, regParam=0.1, rank=25, userCol="Reviewer ID", itemCol="Hotel ID", ratingCol="Score", coldStartStrategy="drop", nonnegative=True)
model_als = als.fit(training)

In [10]:
# Đánh giá mô hình
predictions = model_als.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")
predictions.show(5)

Root-mean-square error = 0.9703634889601636
+--------+-----------+-----+----------+
|Hotel ID|Reviewer ID|Score|prediction|
+--------+-----------+-----+----------+
|      15|        148|  9.6|  9.276711|
|      17|        148| 10.0|  9.372315|
|       4|         31| 10.0|  8.774868|
|       5|         31| 10.0|  8.883228|
|      14|         31| 10.0| 8.9721365|
+--------+-----------+-----+----------+
only showing top 5 rows



In [12]:
# Kiểm tra tên các cột trong hotel_info_df
print(hotel_info_df.columns)

Index(['num', 'Hotel_ID', 'Hotel_Name', 'Hotel_Rank', 'Hotel_Address',
       'Total_Score', 'Location', 'Cleanliness', 'Service', 'Facilities',
       'Value_for_money', 'Comfort_and_room_quality', 'comments_count',
       'Hotel_Description', 'Hotel_Rank_Num'],
      dtype='object')


In [22]:
def get_hotel_recommendations_for_user(reviewer_id, num_recommendations=5):
    # Tạo DataFrame cho Reviewer ID
    reviewer_df = spark.createDataFrame([(reviewer_id,)], ["Reviewer ID"])
    
    # Tạo các gợi ý cho Reviewer ID
    user_recommendations = model_als.recommendForUserSubset(reviewer_df, num_recommendations)
    
    
    # Hiển thị kết quả
    recommendations = user_recommendations.collect()[0].asDict()['recommendations']
    
    # Lấy ID khách sạn được gợi ý
    hotel_ids = [rec['Hotel ID'] for rec in recommendations]
    

    # Chuyển đổi 'Hotel_ID' trong hotel_info_df thành kiểu int nếu cần
    hotel_info_df['Hotel_ID'] = hotel_info_df['Hotel_ID'].astype(int)
    
    # Lấy thông tin chi tiết của các khách sạn được gợi ý (sử dụng 'Hotel_ID')
    recommended_hotels = hotel_info_df[hotel_info_df['Hotel_ID'].isin(hotel_ids)]
    
    
    return recommended_hotels[['Hotel_ID', 'Hotel_Name', 'Hotel_Rank', 'Hotel_Address', 'Total_Score']]

# Ví dụ: Đưa ra gợi ý khách sạn cho Reviewer ID = 1
recommended_hotels = get_hotel_recommendations_for_user(1)
print(recommended_hotels)


    Hotel_ID                                         Hotel_Name    Hotel_Rank  \
0         11  Khách sạn Mường Thanh Luxury Nha Trang (Muong ...  5 sao trên 5   
1         12                               ALPHA BIRD NHA TRANG  4 sao trên 5   
31        22                  Muong Thanh Grand Nha Trang Hotel  4 sao trên 5   
32        23                              ELITE HOTEL NHA TRANG  4 sao trên 5   

                                        Hotel_Address  Total_Score  
0           60 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam          8.8  
1   51/19/37 Tue Tinh St, Loc Tho Ward, Nha Trang,...          7.7  
31  Số 6 Dương Hiến Quyền, Vĩnh Hòa, Vĩnh Hòa, Nha...          8.6  
32  50 Đ. Củ Chi, Vĩnh Hải, ELite Hotel Nha Trang,...          9.7  


In [23]:
# Lưu mô hình ALS đã huấn luyện
model_path = "als_model"
model_als.save(model_path)